In [1]:
%cd ../..
%pwd
# Should be  <BASE>/code

/home/oli/Research/Joe/agent-goals/code


'/home/oli/Research/Joe/agent-goals/code'

# Random Experiments

The idea here is to generate a lot of PDGs, perform inference on them in different ways / with different parameters, and report (1) timing in formation and (2) training curves. 

Inference algorithms on PDGs to pit against each other:
 * `ExpCone` on full joint distribution;
 * `ExpCone` on clique trees (automatically discovered);
 * `Pytorch` (different learning rates, optimizers: `LBFGS`; `ADAM`)
 * LIR (still needs implementation + schedule)

Of course, we also need to compare against known algorithms; we can generate / use as benchmarks. 
For examples of BNs, I can use

* http://constantinou.info/downloads/bayesys/bayesys_repository.pdf
* the pgmpy example library
* randomly generated BNs

But first some imports...

In [3]:
from pdg.pdg import PDG
from pdg.rv import Variable as Var, Unit
from pdg.dist import RawJointDist as RJD, CPT

from random import randint, sample

import numpy as np
import networkx as nx
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor

from functools import reduce
from operator import and_


## 1. Make some random BNs.

## 3. Repeat For Factor Graphs!
Start by making some random factor graphs.

In [7]:

NVARS = 10
NFGs = 15
MIN_FACTORS = 2
MAX_FACTORS = 20

varis = [ Var.alph("X%d"%i, randint(2,4)) for i in range(NVARS)]


fgs = []
pdgs = []

while len(fgs) < NFGs:
	nfactors = randint(MIN_FACTORS, MAX_FACTORS)
	fg = FactorGraph()
	pdg = PDG()

	for j in range(nfactors):
		selection = sample(varis, randint(1,4))
		f = np.random.rand(*[len(v) for v in selection])
		df = DiscreteFactor(
			[v.name for v in selection],
			[len(v) for v in selection],
			f
		)
		fg.add_nodes_from([v.name for v in selection]) # how silly
		fg.add_factors(df)
		fg.add_edges_from([(v.name, df) for v in selection])

		pdg += CPT.from_matrix(Unit, reduce(and_, selection), f.reshape(1,-1)/f.sum() )
	
	if(nx.is_connected(fg)):	
		fgs.append(fg)
		pdgs.append(pdg)

In [8]:
from pgmpy.inference import BeliefPropagation
from itertools import chain


fg = fgs[0]
assert fg.check_model()


True

In [35]:
gr = nx.Graph(fg.edges())
# [*nx.connected_components(gr)]
# cl = list(nx.clique.find_cliques(gr))
# list(gr.nodes())[1]
gr2 = nx.relabel_nodes(gr, 
    { oldnode : ','.join(oldnode.variables) if hasattr(oldnode,'variables') else oldnode
             for oldnode in gr.nodes()})
# list(nx.clique.find_cliques(gr2))
# len(cl)

In [15]:
bp = BeliefPropagation(fg)
bp.calibrate();

In [18]:
bp.get_cliques()

NodeView((('X7', 'X9'), ('X3', 'X9', 'X4'), ('X3', 'X8', 'X1', 'X4'), ('X3', 'X8', 'X5', 'X0', 'X4'), ('X3', 'X8', 'X5', 'X0', 'X2')))

Now for the PDG inference part.

In [ ]:
from pdg.alg.interior_pt import cvx_opt_clusters
